In [228]:
from underthesea import word_tokenize
from underthesea import ner
from nltk import sent_tokenize
import requests
import framler
import string
from urllib.parse import urlparse
from google import google
import framler
import operator
from tqdm import tqdm

In [229]:
with open('stopwords.txt', 'r') as f:
    stopwords = f.readlines()
    stopwords = stopwords[-1].split()

In [230]:
def tokenize(text):
    sents = sent_tokenize(text)
    sents = [word_tokenize(s,format = 'text') for s in sents]
    return sents

def keywords_extraction(sentences):
    sent = sentences.lower()
    sent = sent.split()
    sent = [s for s in sent if s not in stopwords and s not in string.punctuation]
    return sent

In [245]:
question = "Người yêu Hoàng Kiều là ai?"
question_tokenized = tokenize(question)
keywords = keywords_extraction(" ".join(question_tokenized))

In [246]:
keywords

['người_yêu', 'hoàng_kiều', 'ai']

In [247]:
search_result = google.search(question, 1)

In [248]:
domain_dict = {
    "vi.wikipedia.org": "wiki",
    "vnexpress.net": "vnexpress",
    "news.zing.vn": "zingnews",
    "dantri.com.vn": "dantri",
    "thethao.thanhnien.vn": "thethaothanhnien",
    "vietnamnet.vn": "vietnamnet",
    "tuoitre.vn": "tuoitrevn",
    "thanhnien.vn": "thanhnien",
    "cafebiz.vn": "cafebiz",
    "laodong.vn": "laodong"
}

In [249]:
result = {}

for i in tqdm(range(len(search_result))):
    url = search_result[i].link
    domain = urlparse(url).netloc
    if domain not in domain_dict.keys():
        continue
    domain = domain_dict[domain]
    
    dt = framler.NewspapersParser(domain)
    print(url)
    text = dt.parse(url).text
    
    passages = []

    for sent in text.split('.'):
        sent_tok = " ".join(tokenize(sent))
        sent_tok = sent_tok.replace('\n', '').strip().lower()
        sent_keywords = keywords_extraction(sent_tok)
        num_overlap_keywords = len(set(sent_keywords) & set(keywords))
        if num_overlap_keywords > 0:
            passages.append(sent)
    
    for p in passages:
        res = []
        for info in ner(p):
            if info[3] == 'B-PER':
                res.append(info[0])
            if info[3] == 'I-PER':
                res[-1] += ' ' + info[0]
        for r in res:
            if r in result.keys():
                result[r] += 1
            else:
                result[r] = 1
    for key in result.keys():
        if "_".join(key.split()).lower() in keywords: #Remove keyword from question
            result[key] = 0


  0%|          | 0/8 [00:00<?, ?it/s]

https://vi.wikipedia.org/wiki/Ho%C3%A0ng_Ki%E1%BB%81u



 12%|█▎        | 1/8 [00:00<00:04,  1.52it/s]

https://vietnamnet.vn/hoang-kieu-tag27907.html



100%|██████████| 8/8 [00:01<00:00,  4.95it/s]

In [250]:
sorted_result = sorted(result.items(), key=operator.itemgetter(1), reverse=True)

In [251]:
sorted_result[:5]

[('RAAS', 1), ('Ngọc Trinh', 1), ('Hoàng Kiều', 0)]